In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome() 

driver.get("https://careersmw.com/")

In [ ]:
def scrape_listings(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    job_listings = soup.find_all('li', class_='job_listing')
    listings_data = []
    for listing in job_listings:
        job_position_elem = listing.find('h3')
        job_position = job_position_elem.text.strip() if job_position_elem else None

        company_elem = listing.find('div', class_='company')
        company = company_elem.strong.text.strip() if company_elem else None

        location_elem = listing.find('div', class_='location')
        location = location_elem.text.strip() if location_elem else None

        job_type_elem = listing.find('li', class_='job-type')
        job_type = job_type_elem.text.strip() if job_type_elem else None

        posting_date_elem = listing.find('li', class_='date')
        posting_date = posting_date_elem.time['datetime'] if posting_date_elem else None

        if job_position and company and location and job_type and posting_date:
            listing_data = {
                'Position': job_position,
                'Company': company,
                'Location': location,
                'Job Type': job_type,
                'Posting Date': posting_date
            }
            listings_data.append(listing_data)
    return listings_data


while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "load_more_jobs"))
        )
        load_more_button.click()
        time.sleep(2)  
        page_source = driver.page_source
        new_listings = scrape_listings(page_source)
        if not new_listings:
            break
    except:
        break  


driver.quit()

In [ ]:
df = pd.DataFrame(new_listings)

print(df)

df.to_csv('job_listings.csv', index=False)